In [1]:
import requests
import json
import pandas as pd
import feedparser as fp
import newspaper as np

def mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories):
    # Liste des nouveaux articles
    new_articles = []

    # Parcourir les flux RSS
    for source_url in flux_rss_a_lire:
        # Déterminer la catégorie du flux
        category = categories[source_url]

        data = fp.parse(source_url)
        
        # Parcourir les articles du flux RSS
        for item in data.entries:
            # Clés renommées pour correspondre à la base de données existante
            article = {
                "title": item.title,
                "date": item.published,  # Renommé de 'published' à 'date'
                "url": item.link,  # Renommé de 'link' à 'url'
                "content": item.description if "description" in item else "",
                "image_link": item.enclosures[0].url if "enclosures" in item and item.enclosures[0].type == "image/jpeg" else "",
                "category": category,  # Ajouté la catégorie
            }
            
            # Vérifier si l'article existe déjà dans la base de données
            if article["url"] not in database:
                article_obj = np.Article(article["url"])
                article_obj.download()
                article_obj.parse()
                
                # Mettre à jour les informations de l'article
                article["content"] += "\n\n" + article_obj.text  # Ajouter le contenu téléchargé
                article["author"] = article_obj.authors  # Ajouté la liste des auteurs
                
                # Ajouter l'article à la liste des nouveaux articles
                new_articles.append((article["url"], article))

    # Mettre à jour la base de données existante avec les nouveaux articles
    for url, article in new_articles:
        database[url] = article

    return database


# Lien vers le fichier tvinfo-sources.json
tvinfo_sources_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json"

# Charger la liste des flux RSS depuis le lien tvinfo-sources.json
response_sources = requests.get(tvinfo_sources_url)
sources = json.loads(response_sources.text)

# Liste des flux RSS à parcourir
flux_rss_a_lire = list(sources.values())

# Usage de la fonction avec les catégories
categories = {
    "https://www.francetvinfo.fr/france.rss": "france",
    "https://www.francetvinfo.fr/monde/europe.rss": "europe",  # Corrigé ici
    "https://www.francetvinfo.fr/economie/entreprises.rss": "economie"
}


# Charger la base de données existante depuis le lien francetvinfo.json
francetvinfo_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json"
response_database = requests.get(francetvinfo_url)
database = json.loads(response_database.text)

# Mettre à jour la base de données avec les nouveaux articles
database_mise_a_jour = mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories)

# Transformer la base de données mise à jour en DataFrame
df_database_mise_a_jour = pd.DataFrame(database_mise_a_jour.values())

# Ajouter les URLs comme colonne dans le DataFrame
df_database_mise_a_jour['url'] = list(database_mise_a_jour.keys())

# Afficher les premières lignes pour vérifier
df_database_mise_a_jour





,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,NaN,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,NaN,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,NaN,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,NaN,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,NaN,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2152,Stellantis : la rémunération de Carlos Tavares...,"Fri, 23 Feb 2024 17:16:54 +0100",[Franceinfo Avec Afp],economie,Les revenus du patron du constructeur automobi...,https://www.francetvinfo.fr/pictures/QObleFnov...,NaN,https://www.francetvinfo.fr/economie/stellanti...
2153,"Colère des agriculteurs : le ""roi du poulet"" u...","Fri, 23 Feb 2024 12:01:04 +0100",[France],economie,Emmanuel Macron estime que la suspension des d...,https://www.francetvinfo.fr/pictures/1sIyGZLEG...,NaN,https://www.francetvinfo.fr/economie/crise/blo...
2154,C'est quoi le CAC 40 ?,"Fri, 23 Feb 2024 10:53:26 +0100",[Gérald Roux],economie,"La Bourse de Paris cartonne, le CAC 40 a battu...",https://www.francetvinfo.fr/pictures/Mshir-Lj4...,NaN,https://www.francetvinfo.fr/replay-radio/expli...
2155,Le premier cobaye humain peut contrôler une so...,"Thu, 22 Feb 2024 13:26:19 +0100",[Marie Dupin],economie,"La société Neuralink, l’une des nombreuses soc...",https://www.francetvinfo.fr/pictures/ZpMQGPoia...,NaN,https://www.francetvinfo.fr/replay-radio/dans-...


# Mettre à jour la base de données de Git

In [2]:

# Fonction pour récupérer la base de données actuelle depuis GitHub
def telecharger_base_donnees_github(url):
    response = requests.get(url)
    data = response.text.splitlines()  # Séparer chaque ligne du fichier JSON
    df = pd.DataFrame([pd.read_json(line, typ='series') for line in data])
    return df

# Fonction pour concaténer l'ancienne base de données avec les nouveaux articles
def concatener_bases_donnees(df_ancienne, df_nouveaux_articles):
    # Filtrer les nouveaux articles pour ne garder que ceux qui n'existent pas déjà dans l'ancienne base de données
    nouveaux_articles_uniques = df_nouveaux_articles[~df_nouveaux_articles['url'].isin(df_ancienne['url'])]
    # Concaténer les deux DataFrames
    df_mise_a_jour = pd.concat([df_ancienne, nouveaux_articles_uniques], ignore_index=True)
    return df_mise_a_jour

# URL de la base de données sur GitHub
url_github = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'

# Charger la base de données actuelle
df_ancienne = telecharger_base_donnees_github(url_github)

# Concaténer l'ancienne base de données avec les nouveaux articles
df_mise_a_jour = concatener_bases_donnees(df_ancienne, df_database_mise_a_jour)

# Sauvegarder la base de données mise à jour localement
df_mise_a_jour.to_json("Data/francetvinfo_maj.json", orient="records", lines=True, force_ascii=False)
df_mise_a_jour.to_csv("Data/francetvinfo_maj.csv", index=False)


In [3]:
df_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2518,"REPORTAGE. ""On veut des prix dignes, qui nous ...","Wed, 28 Feb 2024 16:01:56 +0100",[France],europe,Les ministres de l'Agriculture polonais et ukr...,https://www.francetvinfo.fr/pictures/4eJ7Ylx_V...,NaN,https://www.francetvinfo.fr/economie/crise/blo...
2519,Envoi de troupes en Ukraine : Emmanuel Macron ...,"Wed, 28 Feb 2024 15:58:56 +0100",[],europe,De nombreux pays européens ont fait savoir leu...,,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2520,Entreprises : les TPE en appellent à l'État po...,"Thu, 29 Feb 2024 20:12:01 +0100","[France, Jean-Guilhem Darré]",economie,Les tarifs réglementés de l'électricité vont b...,https://www.francetvinfo.fr/pictures/3mhLQ4H3p...,NaN,https://www.francetvinfo.fr/replay-radio/l-int...
2521,Énergie : des tarifs pour protéger les entrepr...,"Thu, 29 Feb 2024 17:09:05 +0100",[],economie,Les factures d'électricité devraient baisser p...,,NaN,https://www.francetvinfo.fr/economie/energie/e...


In [4]:
import pandas as pd
# URL du fichier mise à jour chaque jour JSON sur GitHub
url = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'
# Charger les données dans un DataFrame
francetvinfo_mise_a_jour = pd.read_json(url, lines=True)
# Afficher les premières lignes du DataFrame
francetvinfo_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2476,Guerre en Ukraine : quelles sont les lignes dé...,"Tue, 27 Feb 2024 21:51:16 +0100",[],europe,À quoi pourrait ressembler l’envoi de troupes ...,,None,https://www.francetvinfo.fr/monde/europe/manif...
2477,Guerre en Ukraine : les propos d'Emmanuel Macr...,"Tue, 27 Feb 2024 20:35:59 +0100",[],europe,"Lundi 26 février, lors d'une conférence de pre...",https://www.francetvinfo.fr/pictures/9XLQZZnXR...,None,https://www.francetvinfo.fr/monde/europe/manif...
2478,Casino : le distributeur a perdu près de 6 mil...,"Wed, 28 Feb 2024 10:00:34 +0100",[Franceinfo Avec Afp],economie,Cette perte spectaculaire est imputable aux gr...,https://www.francetvinfo.fr/pictures/iXjTOvUQk...,None,https://www.francetvinfo.fr/economie/entrepris...
2479,Pouvoir d’achat : pour quelles raisons les PME...,"Wed, 28 Feb 2024 08:20:29 +0100",[Emmanuel Cugny],economie,"Avec un début d’année difficile, les petites, ...",https://www.francetvinfo.fr/pictures/9IffAL0Kg...,None,https://www.francetvinfo.fr/replay-radio/le-br...
